In [ ]:
%load_ext autoreload
%autoreload 2

import os
from georges_core.sequences import TwissSequence
from georges_core import twiss
import zgoubidoo
from zgoubidoo.commands import *
from zgoubidoo import ureg as _

In [ ]:
os.environ['ZGOUBI_EXECUTABLE_PATH'] = "/Users/rtesse/packages/zgoubi-build-clion-develop_gnu9/install/bin/"

# Load the MAD-X Twiss table and convert to Zgoubi input file

In [ ]:
input_madx = TwissSequence(filename=f"test45degspreader.outx")
zi = zgoubidoo.Input.from_sequence(sequence=input_madx,
                                   beam=BeamInputDistribution,
                                   with_survey=True,
                                   with_survey_reference=True,
                                   converters={'sbend': zgoubidoo.converters.sbend_to_zgoubi,
                                               'quadrupole': zgoubidoo.converters.quadrupole_to_zgoubi},
                                   options={
                                       'sbend': {'command': Dipole}
                                   }
                                  )
zi.XPAS = 5*_.cm
zr_tracking = zgoubidoo.Zgoubi()(zi).collect()

# Make the Twiss

In [ ]:
twiss_beam = Objet5('BEAM', BORO=input_madx.kinematics.brho)

In [ ]:
zi.replace('BEAM', twiss_beam)
zi.XPAS=15*_.cm

zr_twiss = zgoubidoo.Zgoubi()(zi).collect()
M = zgoubidoo.twiss.compute_transfer_matrix(
    beamline=zi,
    tracks=zr_twiss.tracks_frenet)
results_twiss = twiss.Twiss(twiss_init=input_madx.betablock)(matrix=M)

# Plotting

## Global coordinates

In [ ]:
artist=zgoubidoo.vis.ZgoubidooPlotlyArtist(width=1200)

artist.fig['layout']['xaxis']['title'] = 'X (m)'
artist.fig['layout']['yaxis']['title'] = 'Y (m)'

artist.scatter(
    x=zr_tracking.tracks_global['XG'], 
    y=zr_tracking.tracks_global['YG'],
    marker={'color': 'red', 'size': 2},
    mode='lines',
    showlegend=False,
    )

artist.plot_beamline(beamline=zi, with_frames=True, with_apertures=True, with_map=True)
artist.render()

## Twiss

### Beta function

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist()
artist.fig['layout']['xaxis']['title'] = r'S (m)'
artist.fig['layout']['yaxis']['title'] = r'$\beta (m)$'
artist.fig['layout']['width'] = 1200
artist.fig['layout']['height'] = 800

artist.scatter(
    x=results_twiss['S'],
    y=results_twiss['BETA11'],
    marker={'color': 'darkblue', 'size': 2},
    mode='lines',
    name=f"Horizontal",
)

artist.scatter(
    x=results_twiss['S'], 
    y=results_twiss['BETA22'],
    marker={'color': 'darkred', 'size': 2},
    mode='lines',
    name=f"Vertical",
    )


artist.scatter(
    x=input_madx.df['S'],
    y=input_madx.df['BETX'],
    marker={'color': 'black', 'symbol': '303'},
    mode='markers',
    name=f"MAD-X",
    showlegend = True
)

artist.scatter(
    x=input_madx.df['S'],
    y=input_madx.df['BETY'],
    marker={'color': 'black', 'symbol': '303'},
    mode='markers',
    name=f"MAD-X",
    showlegend = False
)


artist.plot_cartouche(beamline=zi, vertical_position=1.12)
artist.render()

### Dispersion

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist()
artist.fig['layout']['xaxis']['title'] = r'S (m)'
artist.fig['layout']['yaxis']['title'] = r'$D_x(m)$'
artist.fig['layout']['width'] = 1200
artist.fig['layout']['height'] = 800

artist.scatter(
    x=results_twiss['S'],
    y=results_twiss['DISP1'],
    marker={'color': 'darkorange', 'size': 2},
    mode='lines',
    name=f"Horizontal",
)


artist.scatter(
    x=input_madx.df['S'],
    y=input_madx.df['DX'],
    marker={'color': 'black', 'symbol': '303'},
    mode='markers',
    name=f"MAD-X",
    showlegend = True
)


artist.plot_cartouche(beamline=zi, vertical_position=1.12)
artist.render()